# Assistant-05<br/>Code Interpreter & Function calling

## Trading Assistant that can calculate portfolio values and email a report

## Notes

- 01/14/2024: Uploading a CSV file failed

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

## Email functionality

- Implemented using Logic Apps with an Http POST trigger

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [8]:
import time
import oaihelper as framework
import yfinance as yf


## Get an OpenAI client
client = framework.get_openai_client(api_key=framework.api_KEY,
        api_version=framework.api_version,
        azure_endpoint=framework.api_URI,)

### Get the latest stock price by ticker symbol using Yahoo Finance

In [9]:
def get_stock_price(symbol: str) -> float:
    try:
        stock = yf.Ticker(symbol)
        price = stock.history(period="1d")['Close'].iloc[-1]    
        return price
    except:
        return 0

### Prepare the tools for function calling

In [10]:
tools_list = [
{"type": "code_interpreter"},
{"type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "send_email",
        "description": "Sends an email to a recipient(s).",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The email(s) the email should be sent to."
                },
                "content": {
                    "type": "string",
                    "description": "The content of the email."
                }
            },
            "required": ["to", "content"]
        }
    }
}]

In [11]:
csv_file = framework.upload_file(client,"../data/fta-portfolio.csv")

### Create the assistant

In [12]:
framework.clear_shelves()

## Create an assistant
stock_assistant = framework.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list,                                                  
                                                  model=framework.gpt_deployment_name,
                                                  file_ids=[csv_file.id]
                                                  )

Added assistant:  asst_kyfLQ7ioPqs0faC1YvFYPPaN 2


### Create a thread, put a message on the thread, and view the initial status

In [13]:
# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Based on the provided portfolio, please email me an HTML formatted report to alemor@microsoft.com with the details for each stock based on the latest stock prices, and list the best and worst performing stocks in my portfolio."
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=stock_assistant.id,
    instructions="Today's date and time is: " + framework.get_localized_datetime() + ".\n\n"
)

#print(run.model_dump_json(indent=4))

### Process the message, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [14]:
while True:
    # Wait for 1 seconds
    time.sleep(5)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    #print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        ## Cleanup
        print(client.beta.threads.delete(thread.id))
        print(client.beta.assistants.delete(stock_assistant.id))
        print(client.files.delete(csv_file.id))

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output
                })
            elif func_name == "send_email":
                print("Sending email...")
                email_to = arguments['to']
                email_content = arguments['content']
                framework.send_email({'to': email_to,'content':email_content})
                
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": "Email sent"
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

Waiting for the Assistant to process...
Waiting for the Assistant to process...
Function Calling
{'tool_calls': [{'id': 'call_MDf3RDJVBumRHoH99Trn2EVe', 'function': {'arguments': '{"symbol": "AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_1jfzdD8XWpxvFxUZhbwH66yu', 'function': {'arguments': '{"symbol": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_KNGFTnOkVIeZ7NE0YgRwlzPR', 'function': {'arguments': '{"symbol": "GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_tf8PsATH3hXHek7IPRKkls8G', 'function': {'arguments': '{"symbol": "AMZN"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_U2nTfN1xOfvwQwxpL47EhdFh', 'function': {'arguments': '{"symbol": "TSLA"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_E0apT6RBBnkiVQJ825lIOBY7', 'function': {'arguments': '{"symbol": "NFLX"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_whqcoq5wSsSsnreAAAuicz92', 'function': {'arguments'